In [1]:
"""
Code based on:
Shang et al "Edge Attention-based Multi-Relational Graph Convolutional Networks" -> https://github.com/Luckick/EAGCN
Coley et al "Convolutional Embedding of Attributed Molecular Graphs for Physical Property Prediction" -> https://github.com/connorcoley/conv_qsar_fast
"""
import os
import logging
import pickle

import numpy as np
import pandas as pd
import torch
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import MolFromSmiles
from sklearn.metrics import pairwise_distances
from torch.utils.data import Dataset

use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
IntTensor = torch.cuda.IntTensor if use_cuda else torch.IntTensor
DoubleTensor = torch.cuda.DoubleTensor if use_cuda else torch.DoubleTensor


def load_data_from_smiles(smiles, add_dummy_node=True, one_hot_formal_charge=True):
    """Load and featurize data from lists of SMILES strings and labels.

    Args:
        x_smiles (list[str]): A list of SMILES strings.
        labels (list[float]): A list of the corresponding labels.
        add_dummy_node (bool): If True, a dummy node will be added to the molecular graph. Defaults to True.
        one_hot_formal_charge (bool): If True, formal charges on atoms are one-hot encoded. Defaults to False.

    Returns:
        A tuple (X, y) in which X is a list of graph descriptors (node features, adjacency matrices, distance matrices),
        and y is a list of the corresponding labels.
    """

    try:
        mol = MolFromSmiles(smiles)
        try:
            mol = Chem.AddHs(mol)
            AllChem.EmbedMolecule(mol, maxAttempts=5000)
            AllChem.UFFOptimizeMolecule(mol)
            mol = Chem.RemoveHs(mol)
        except:
            AllChem.Compute2DCoords(mol)
        afm, adj, dist = featurize_mol(mol, add_dummy_node, one_hot_formal_charge)

    except ValueError as e:
        print('the SMILES ({}) can not be converted to a graph.\nREASON: {}'.format(smiles, e))

    return afm, adj, dist


def featurize_mol(mol, add_dummy_node, one_hot_formal_charge):
    """Featurize molecule.

    Args:
        mol (rdchem.Mol): An RDKit Mol object.
        add_dummy_node (bool): If True, a dummy node will be added to the molecular graph.
        one_hot_formal_charge (bool): If True, formal charges on atoms are one-hot encoded.

    Returns:
        A tuple of molecular graph descriptors (node features, adjacency matrix, distance matrix).
    """
    node_features = np.array([get_atom_features(atom, one_hot_formal_charge)
                              for atom in mol.GetAtoms()])
    #print(node_features.shape)
    adj_matrix = np.eye(mol.GetNumAtoms())
    for bond in mol.GetBonds():
        begin_atom = bond.GetBeginAtom().GetIdx()
        end_atom = bond.GetEndAtom().GetIdx()
        adj_matrix[begin_atom, end_atom] = adj_matrix[end_atom, begin_atom] = 1

    conf = mol.GetConformer()
    pos_matrix = np.array([[conf.GetAtomPosition(k).x, conf.GetAtomPosition(k).y, conf.GetAtomPosition(k).z]
                           for k in range(mol.GetNumAtoms())])
    dist_matrix = pairwise_distances(pos_matrix)

    if add_dummy_node:
        m = np.zeros((node_features.shape[0] + 1, node_features.shape[1] + 1))
        m[1:, 1:] = node_features
        m[0, 0] = 1.
        node_features = m

        m = np.zeros((adj_matrix.shape[0] + 1, adj_matrix.shape[1] + 1))
        m[1:, 1:] = adj_matrix
        adj_matrix = m

        m = np.full((dist_matrix.shape[0] + 1, dist_matrix.shape[1] + 1), 1e6)
        m[1:, 1:] = dist_matrix
        dist_matrix = m

    return node_features, adj_matrix, dist_matrix


def get_atom_features(atom, one_hot_formal_charge=True):
    """Calculate atom features.

    Args:
        atom (rdchem.Atom): An RDKit Atom object.
        one_hot_formal_charge (bool): If True, formal charges on atoms are one-hot encoded.

    Returns:
        A 1-dimensional array (ndarray) of atom features.
    """
    attributes = []

    attributes += one_hot_vector(
        atom.GetAtomicNum(),
        #[6, 8, 7, 16, 35, 19, 53, 11, 17, 9, 15, 46, 5, 14, 55, 1, 13, 3, 26, 24, 22, 12, 25, 29,
        # 44, 28, 78, 47, 30, 75, 56, 50, 27, 20, 34, 45, 58, 40, 80, 74, 70, 21, 999]
        [5, 6, 7, 8, 9, 15, 16, 17, 35, 53, 999]
        #[6, 7, 8, 9, 15, 16, 17, 35, 53]
        #[9, 6, 17, 7, 15, 8, 35, 53, 16]
    )

    attributes += one_hot_vector(
        len(atom.GetNeighbors()),
        [0, 1, 2, 3, 4, 5]
    )

    attributes += one_hot_vector(
        atom.GetTotalNumHs(),
        [0, 1, 2, 3, 4]
    )

    if one_hot_formal_charge:
        attributes += one_hot_vector(
            atom.GetFormalCharge(),
            [-1, 0, 1]
        )
    else:
        attributes.append(atom.GetFormalCharge())

    attributes.append(atom.IsInRing())
    attributes.append(atom.GetIsAromatic())
    return np.array(attributes, dtype=np.float32)


def one_hot_vector(val, lst):
    """Converts a value to a one-hot vector based on options in lst"""
    if val not in lst:
        val = lst[-1]
    return map(lambda x: x == val, lst)


#dimension 28

/users/PCON0041/xiaohu/.local/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = pd.read_csv('./USPTO500MT_train_processed_100.csv')
yie = df['yield'].values
print(yie.shape[0])
print(len(np.where(yie<=40.0)[0]) / yie.shape[0])

78201
0.18086725233692663


In [3]:

from rdkit import RDLogger 
RDLogger.DisableLog('rdApp.*')
df = pd.read_csv('./USPTO500MT_train_processed_100.csv')
r_dic = []
ri_list = []
ri_afm_list = []
ri_adj_list = []
ri_dist_list = []
reaction_index_list = []
molecule_index_list = []
yield_list = []

for reaction_index in range(df.shape[0]):
    if reaction_index % 1000 == 0:
        print(reaction_index)
    r = df['reagents'].values[reaction_index]
    for ri_index in range(len(r.split('.'))):
        ri = r.split('.')[ri_index]
        ri_afm, ri_adj, ri_dist = load_data_from_smiles(ri)
        ri_list.append(ri)
        ri_afm_list.append(ri_afm)
        ri_adj_list.append(ri_adj)
        ri_dist_list.append(ri_dist)
        
        reaction_index_list.append(int(reaction_index))
        molecule_index_list.append(int(ri_index))
        yield_list.append(df['yield'].values[reaction_index])
    r_dic = {'reagents': ri_list, 'reagent_afm': ri_afm_list, 'reagent_adj': ri_adj_list, 'reagent_dist': ri_dist_list, 'reaction_index':reaction_index_list, 'molecule_index': molecule_index_list, 'yield': yield_list}
#print(r_dic)      
with open('USPTO500MT_train_processed_100_reagents_feats.pkl', 'wb') as f:
    pickle.dump(r_dic, f)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
